In [1]:
import pandas as pd
import os
import glob
import traceback
from datetime import datetime
import pandas as pd
from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
plotly.offline.init_notebook_mode(connected=True)
import plotly.express as px
import gezi
from gezi import tqdm 
import pymp
import time
import qgrid
from sklearn.metrics import roc_auc_score
from multiprocessing import Manager, cpu_count
from projects.feed.rank.utils.doc_info.KV import KV

In [2]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:10000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [60]:
info_dir = f'/search/odin/publicData/CloudS/mkyuwen/rank/infos/video/11/'
info_dir = f'/search/odin/publicData/CloudS/libowei/rank_online/infos/video/8'
hour = 2020051909
info_path = f'{info_dir}/{hour}/valid.csv'
print(info_path)
df = pd.read_csv(info_path)
# df2 = pd.read_csv()
# df = df.rename({'duration':'dur'})
df['dur'] = df.duration
# df['mark'] = 'video'
# info_dir = f'/search/odin/publicData/CloudS/libowei/rank_online/infos/video/8'
# info_path = f'{info_dir}/{hour}/valid.csv'
# df2 = pd.read_csv(info_path)
# df2['dur'] = df.duration
# df2['mark'] = 'video'
# df = pd.concat([df, df2])
df = df[df.product_data=='sgsapp']
df.head()

/search/odin/publicData/CloudS/libowei/rank_online/infos/video/8/2020051909/valid.csv


,doc_topic,abtestid,article_page_time,coldstart_refresh_num,did,distribution,docid,duration,id,impression_time,...,video_time,click,weight,cold,product_data,pred,pred_click,pred_dur,hour,dur
0,0,5,1588853600,1,-5487410083488566407,5007,20507n0K3H1Y38,0,0523864885048470040\t20507n0K3H1Y38,1589851063,...,49,0,1.0,0,sgsapp,0.06440441,0.50721040,0.07013884,2020051909,0
1,0,12,1589699926,3,-4910287128900267240,3039,20517h0K3yJEod,0,5897862265045219645\t20517h0K3yJEod,1589848549,...,690,0,1.0,0,sgsapp,0.01576874,0.07992744,0.29058588,2020051909,0
2,0,6,1588667226,1,7297875804759759819,3039,20505h040MSZoE,0,78eb868198043360265\t20505h040MSZoE,1589849772,...,129,0,1.0,0,sgsapp,0.00354569,0.05164549,0.07756320,2020051909,0
3,0,7,1589450501,8,-2011539449682497723,AisiAssist,20514j0K0bE7J7,332,cb87f9e26e787cec2f3233b3b769a1ffd58bd689cef3\t...,1589850237,...,332,1,1.0,0,sgsapp,0.01853448,0.06698492,0.50817640,2020051909,332
4,0,14,1589443825,2,3101413342895488890,1445,20514g0Q0lFEbp,58,f460862363030743206\t20514g0Q0lFEbp,1589849317,...,58,1,1.0,0,sgsapp,0.05921218,0.39634532,0.09834132,2020051909,58


In [61]:
df['click'] = (df.dur != 0).astype(int)

In [75]:
df['impression'] = df['mid'] + '\t' + df['impression_time'].astype('str')

In [76]:
df5 = df[df.abtestid==5]

In [77]:
df5.video_time.mean()

307.47042998064853

In [78]:
df5_ = df5.docid.value_counts().reset_index().head(20)
# df5_['index']

In [79]:
df8 = df[df.abtestid==8]
df8.video_time.mean()

302.4304493513188

In [80]:
df8_ = df8.docid.value_counts().reset_index().head(20)
# df8_['index']

In [81]:
len(set(df5.docid.values)) / len(set(df5.mid.values))

4.080506558118499

In [82]:
len(set(df8.docid.values)) / len(set(df8.mid.values))

4.22217555648887

In [83]:
from projects.feed.rank.utils.doc_info.KV import KV
kv = KV()
titles = kv.get_titles(list(set(df5.docid.values)))

get_titles: 100%|##########| 91/91 [00:05<00:00, 15.72it/s]


In [84]:
df5_ = df5.docid.value_counts().reset_index().head(20)
df5_['title'] = df5_['index'].apply(lambda x: titles.get(x, ''))

In [85]:
df15 = df[df.abtestid==15]

In [86]:
df15.groupby('mid', as_index=False)['impression'].nunique().mean()

2.616373767681097

In [87]:
# df15[df15.mid=='e86a869043036649853']

In [88]:
df15.groupby('mid', as_index=False)['impression'].count().mean()

impression    9.78825547
dtype: float64

In [108]:
df15.groupby('mid', as_index=False)['impression'].nunique().max()

67

In [109]:
df5.groupby('mid', as_index=False)['impression'].nunique().max()

35

In [94]:
df5 = df[df.abtestid==5]
# df5.groupby('mid', as_index=False)['impression'].count().sort_values('impression')

In [95]:
df5.groupby('mid', as_index=False)['impression'].nunique().mean()

2.563545906829489

In [104]:
df5.groupby('mid', as_index=False)['impression'].count().mean()

impression    9.58254184
dtype: float64

In [102]:
df5.groupby('mid', as_index=False)['impression'].count().sort_values('impression')

,mid,impression
1771,cc4e862493032493623,1
949,6a2a1587102449506|c4e4a3,1
1817,d430869007045255051,1
950,6a3e861945030222941,1
1516,ad6aA00000644BD511,1
...,...,...
111,0ae5866708041992158,108
977,6deb868515036995387,110
662,4c84867519037314201,152
1255,8dcd866841039413893,160


In [96]:
len(set(df5.impression.values))

5668

In [97]:
len(set(df15.impression.values))

6104

In [98]:
time.localtime(df5.impression_time.values[0])

time.struct_time(tm_year=2020, tm_mon=5, tm_mday=19, tm_hour=9, tm_min=17, tm_sec=43, tm_wday=1, tm_yday=140, tm_isdst=0)

In [ ]:
df5_

In [ ]:
df8_ = df8.docid.value_counts().reset_index().head(20)
df8_['title'] = df8_['index'].apply(lambda x: titles.get(x, ''))

In [17]:
df8_

,index,docid,title
0,20518h0K0YlGJ5,335,“鸡眼”熟了，看的人直打冷颤。这脚是不是已经成木头了
1,20517f0K3eXQwB,258,Asherah Swim SS 2020迈阿密游泳周游泳时装秀全场
2,2051030435BuEp,193,珍贵影像：“东莞扫黄”前最牛的酒店，每晚上演啥肮脏一幕?
3,20426h043PkRXL,158,眼看大货车就要掉下大桥，后面司机一个举动，成功挽救一个家庭
4,20510d0Q1ws3GD,157,女生为什么都穿“三角裤”，不穿“四角裤”呢?看完总算明白了!
5,20518b0K3bZ5Gx,155,大丈夫：女子带男友参加同学聚会，教授醋意大发，厕所霸王硬上弓
6,20518e0K18jLj2,153,陈赫小品《魂斗罗》陈赫：下次打我直接一点，不要问那么多问题
7,20427d041tAHJu,150,人去世后埋进土里，3D动画模拟身体的变化，看完不知不觉有点心酸!
8,2051890K2wrHDo,112,经典：兄弟俩下棋，一盘棋竟下了1千年，周围的天兵都已变成白骨
9,20506n043ir3fp,112,小伙喜欢上年轻的单亲妈妈，通过不懈努力后，终于得偿所愿了


In [18]:
pd.set_option('display.max_rows', 1000)
pd.set_option("display.precision", 8)

In [19]:
if 'article_page_time' not in df.columns:
  df.article_page_time = df.impression_time
df['page_time'] = (df.impression_time - df.article_page_time) / 60 / 60 / 24

In [20]:
df['impression'] = df.mid + '\t' + df.impression_time.astype(str)

In [21]:
mean_duration = df[df.dur > 0].dur.mean()

In [22]:
mean_duration

107.46830833885035

In [23]:
from projects.feed.rank.src import util
df['cold'] = df.rea.apply(lambda x: int(x) in util.cold_starts)
# df = df[df.product_data=='sgsapp']

In [24]:
import math
df.pred = df.pred.apply(lambda x: math.sqrt(x))

In [25]:
cold_mids = set(df[df.cold==True].mid.values)

In [26]:
len(set(df.mid.values))

53669

In [27]:
len(cold_mids)

3535

In [28]:
len(set(df.impression.values)) 

132979

In [29]:
len(set(df.mid.values))

53669

In [30]:
len(set(df.impression.values))  / len(set(df.mid.values))

2.4777618364418936

In [31]:
len(set(df[df.dur!=0].mid.values))

38192

In [32]:
len(set(df[df.dur!=0].mid.values)) / len(set(df.mid.values))

0.7116212338593975

In [33]:
from projects.feed.rank.src.evaluate import evaluate_df as eval
import melt 

In [34]:
melt.init_flags()

In [35]:
df[df.abtestid.isin([4,5,6])].abtestid

8         6
12        4
13        5
33        6
42        6
         ..
996871    5
996975    6
996976    5
997184    4
997390    6
Name: abtestid, Length: 64969, dtype: int64

In [36]:
result = eval(df[df.abtestid.isin([4,5,6])], group_by_impression=True, key='ol_pred', eval_click=False, eval_duration=False)
result 

{'auc': 0.7243860246781154,
 'mse': 0.010316976790523428,
 'mae': 0.04297721955828595,
 'time_auc': 0.7134579702509607,
 'mean': 0.03165065127526469,
 'loss/click': 0.5608294484363151,
 'loss/dur': array(0.10662279, dtype=float32),
 'stats/pos_ratio': 0.16144622820114207,
 'vtime_auc': 0.5615608061619735,
 'finish_auc': 0.6608664554532782,
 'rcr_auc': 0.6966054082301374,
 'ptime_auc': 0.48469571731280525,
 'stats/num_instances': 64969,
 'stats/click_ratio': 0.16144622820114207,
 'stats/time_per_show': 0.2643689755660166,
 'stats/time_per_click': 0.029778417832270487,
 'stats/time_per_user': 0.016145192613817253,
 'stats/vtime_per_show': 5.2978459308842005,
 'stats/vtime_per_click': 5.007610189197759,
 'stats/vtime_per_user': 2.7150143266475646,
 'click/time_auc': 0.6227123441507203,
 'click/loss/dur': array(0.5592743, dtype=float32),
 'click/mse': 0.05830152845425757,
 'click/rmse': 0.7585429055623804,
 'click/mae': 0.13547713468790984,
 'click/mean': 0.05625454649715177,
 'group/auc':